<a href="https://colab.research.google.com/github/nicolerdz/Regresion-no-lineal-y-logistica/blob/main/2022_9_Regresion_Logistica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Cargamos librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
#Cargar archivo csv desde seaborn
df2022=pd.read_csv('Datos_limpios2022_desv.csv', index_col= 0)
df2022

,MP,TC,IMPORTE,IVA,RET ISR,RET IVA,TOTAL MX,TOTAL SAT,FECHA,FOLIO,UUID,RFC,PROVEEDOR,TIPO GASTO,DESCRIPCION,MP.1,TIPO,STATUS,POLIZA
0,PPD,1.0,5551.38,888.22,0.0,0.0,6439.60,6439.60,2022-01-01,280108,9DA0E63F-1D47-441F-93D7-C2E05AE4D35E,BLE1512082R9,BEPENSA LEASING S.A. DE C.V.,ARRENDAMIENTO FINANCIERO,AA RENTA EXIGIBLE CL0000006876 18/36 de 03/12/...,PPD,I,Vigente,PE02-ENE22
1,PPD,1.0,5551.98,888.32,0.0,0.0,6440.30,6440.30,2022-01-01,280109,7E00E776-A839-4D57-B291-411109164B2F,BLE1512082R9,BEPENSA LEASING S.A. DE C.V.,ARRENDAMIENTO FINANCIERO,AA RENTA EXIGIBLE CL0000006877 18/36 de 03/12/...,PPD,I,Vigente,PE03-ENE22
2,PUE,1.0,0.00,0.00,0.0,0.0,0.00,0.00,2022-01-01,007817297284,BB6585C1-3B54-4130-AFD0-1867C3764821,BNM840515VB1,"BANCO NACIONAL DE MEXICO, S.A.",COMISION BANCARIA,Servicios de Facturación,PUE,I,Vigente,NaN
3,NaN,1.0,0.00,0.00,0.0,0.0,0.00,0.00,2022-01-01,5595,B5E4B4EB-6C2F-11EC-BB9D-00155D014009,ZEGC530520V4A,CARLOS ENRIQUE ZERMEÑO GUTIERREZ,COMPRAS,Pago,NaN,P,Vigente,NaN
4,NaN,1.0,0.00,0.00,0.0,0.0,0.00,0.00,2022-01-01,5614,BB6756D1-6CCD-11EC-947D-00155D014407,ZEGC530520V4A,CARLOS ENRIQUE ZERMEÑO GUTIERREZ,COMPRAS,Pago,NaN,P,Vigente,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2572,PUE,1.0,0.00,0.00,0.0,0.0,0.00,0.00,2022-12-01,863376,E90DA089-3C3B-45C8-A19C-B8810D222F60,BIN940223KE0,"BANCO INVEX, S.A., INSTITUCION DE BANCA MULTIP...",COMISION BANCARIA,COMISIONES E INTERESES,PUE,I,Vigente,NaN
2573,PUE,1.0,17310.30,516.32,0.0,0.0,17826.62,17826.62,2022-12-01,1092957089,D720ED7C-F370-4236-B269-8873ABB78745,BMN930209927,BANCO MERCANTIL DEL NORTE SA INSTITUCION DE BA...,COMISION BANCARIA,SERVICIOS BANCARIOS,PUE,I,Vigente,NaN
2574,NaN,1.2,0.00,0.00,0.0,0.0,0.00,0.00,2022-12-01,4,0AFD3EB5-F3DD-43EB-B9FA-79D793CFF417,AAE6411055P8,Aesa Articulos Electricos S.A de C.V,COMPRAS,Pago,NaN,P,Vigente,NaN
2575,NaN,1.2,0.00,0.00,0.0,0.0,0.00,0.00,2022-12-01,16,E3CA1DA8-581A-5844-80D2-CF0317B3BBEA,AAE6411055P8,AESA ARTICULOS ELECTRICOS,COMPRAS,Pago,NaN,P,Vigente,NaN


In [ ]:
#Rellenamos nulos
df2022 =df2022.fillna(method="bfill")
df2022 =df2022.fillna(method="ffill")


In [ ]:
df2022.isnull().sum().sum()

0

In [ ]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df2022['TIPO GASTO'])
unico

array(['ALIMENTOS', 'ANTICIPO ', 'ANTICIPO COMPRA AUTO',
       'ARRENDAMIENTO FINANCIERO', 'ARRENDAMIENTO LOCAL', 'CALIBRACIONES',
       'CAPACITACION', 'CASETAS', 'COMISION BANCARIA', 'COMISION MIXTA',
       'COMPRA AUTO ', 'COMPRAS', 'COMPRAS ', 'CREDITO ', 'DERECHOS',
       'ENERGIA ELECTRICA', 'FLETE', 'GASOLINA', 'GASTOS IMPORTACION ',
       'GERENCIA ADMON', 'HONORARIOS CONTABLES', 'HONORARIOS JURIDICOS',
       'HONORARIOS PM', 'HOSPEDAJE', 'IMSS/INFONAVIT',
       'INSTALACION ELECTRICA', 'MAQUILAS', 'MAQUINARIA', 'MEDICOS',
       'MENSAJERIA', 'MONITOREO CAMARAS ADT', 'MTTO COMPUTO',
       'MTTO EQUIPO COMPUTO', 'MTTO LOCAL', 'MTTO MAQUINARIA',
       'MTTO OFICINA ', 'MTTO TRANSPORTE', 'NO DEDUCIBLE', 'PAPELERIA',
       'PAQUETERIA', 'PEAJES', 'PUBLICIDAD', 'RENTA AUTO',
       'RENTA COPIADORA', 'SANITIZACION', 'SEGUROS', 'SUBCONTRATOS',
       'SUELDOS DIRECCION', 'SUSCRIPCIONES', 'TELEFONO', 'TRANSPORTE',
       'UNIFORMES', 'VALES DESPENSA', 'VARIOS'], dtype=objec

In [ ]:
#Convierto una variable a dicotómica 
df2022['TIPO GASTO']= df2022['TIPO GASTO'].replace(['ALIMENTOS', 'ARRENDAMIENTO FINANCIERO', 'ARRENDAMIENTO LOCAL',
       'ATENCION CLIENTES', 'CALIBRACIONES', 'CAPACITACION',
       'COMBUSTIBLE', 'COMISION BANCARIA', 'COMISION VENTA',
       'DERECHOS', 'ENERGIA ELECTRICA', 'FLETES', 'GERENCIA ADMON',
       'GERENCIA VTAS', 'HONORARIOS ADMON', 'HONORARIOS CONTABLES',
       'HONORARIOS PM', 'HOSPEDAJE', 'IMSS/INFONAVIT', 'MAQUILAS GIC',
       'MAQUINARIA', 'MEDICOS', 'MENSAJERIA', 'MOBILIARIO',
       'MTTO COMPUTO', 'MTTO LOCAL', 'MTTO TRANSPORTE', 'NO DEDUCIBLE',
       'PAPELERIA', 'PAQUETERIA', 'PEAJES', 'PUBLICIDAD',
       'REPARACIONES ELECTRICAS', 'SEGUROS', 'SERV FUNERARIO ',
       'SUBCONTRATOS', 'SUSCRIPCIONES', 'TELEFONO', 'TRANSPORTE',
       'UNIFORME', 'VALES DESPENSA'], "OTROS GASTOS")
df2022

,MP,TC,IMPORTE,IVA,RET ISR,RET IVA,TOTAL MX,TOTAL SAT,FECHA,FOLIO,UUID,RFC,PROVEEDOR,TIPO GASTO,DESCRIPCION,MP.1,TIPO,STATUS,POLIZA
0,PPD,1.0,5551.38,888.22,0.0,0.0,6439.60,6439.60,2022-01-01,280108,9DA0E63F-1D47-441F-93D7-C2E05AE4D35E,BLE1512082R9,BEPENSA LEASING S.A. DE C.V.,OTROS GASTOS,AA RENTA EXIGIBLE CL0000006876 18/36 de 03/12/...,PPD,I,Vigente,PE02-ENE22
1,PPD,1.0,5551.98,888.32,0.0,0.0,6440.30,6440.30,2022-01-01,280109,7E00E776-A839-4D57-B291-411109164B2F,BLE1512082R9,BEPENSA LEASING S.A. DE C.V.,OTROS GASTOS,AA RENTA EXIGIBLE CL0000006877 18/36 de 03/12/...,PPD,I,Vigente,PE03-ENE22
2,PUE,1.0,0.00,0.00,0.0,0.0,0.00,0.00,2022-01-01,007817297284,BB6585C1-3B54-4130-AFD0-1867C3764821,BNM840515VB1,"BANCO NACIONAL DE MEXICO, S.A.",OTROS GASTOS,Servicios de Facturación,PUE,I,Vigente,PE01-ENE22
3,PUE,1.0,0.00,0.00,0.0,0.0,0.00,0.00,2022-01-01,5595,B5E4B4EB-6C2F-11EC-BB9D-00155D014009,ZEGC530520V4A,CARLOS ENRIQUE ZERMEÑO GUTIERREZ,COMPRAS,Pago,PUE,P,Vigente,PE01-ENE22
4,PUE,1.0,0.00,0.00,0.0,0.0,0.00,0.00,2022-01-01,5614,BB6756D1-6CCD-11EC-947D-00155D014407,ZEGC530520V4A,CARLOS ENRIQUE ZERMEÑO GUTIERREZ,COMPRAS,Pago,PUE,P,Vigente,PE01-ENE22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2572,PUE,1.0,0.00,0.00,0.0,0.0,0.00,0.00,2022-12-01,863376,E90DA089-3C3B-45C8-A19C-B8810D222F60,BIN940223KE0,"BANCO INVEX, S.A., INSTITUCION DE BANCA MULTIP...",OTROS GASTOS,COMISIONES E INTERESES,PUE,I,Vigente,PD02-DIC22/PE104-ENE23
2573,PUE,1.0,17310.30,516.32,0.0,0.0,17826.62,17826.62,2022-12-01,1092957089,D720ED7C-F370-4236-B269-8873ABB78745,BMN930209927,BANCO MERCANTIL DEL NORTE SA INSTITUCION DE BA...,OTROS GASTOS,SERVICIOS BANCARIOS,PUE,I,Vigente,PD02-DIC22/PE104-ENE23
2574,PUE,1.2,0.00,0.00,0.0,0.0,0.00,0.00,2022-12-01,4,0AFD3EB5-F3DD-43EB-B9FA-79D793CFF417,AAE6411055P8,Aesa Articulos Electricos S.A de C.V,COMPRAS,Pago,PUE,P,Vigente,PD02-DIC22/PE104-ENE23
2575,PUE,1.2,0.00,0.00,0.0,0.0,0.00,0.00,2022-12-01,16,E3CA1DA8-581A-5844-80D2-CF0317B3BBEA,AAE6411055P8,AESA ARTICULOS ELECTRICOS,COMPRAS,Pago,PUE,P,Vigente,PD02-DIC22/PE104-ENE23


In [ ]:
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df2022[['TOTAL MX']]
Var_Dep= df2022['TIPO GASTO']

In [ ]:
#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

In [ ]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

In [ ]:
#Se escalan todos los datos
escalar = StandardScaler()

In [ ]:
#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [ ]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [ ]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [ ]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 
y_pred

array(['COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS',
       'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS',
       'COMPRAS', 'COMPRAS', 'COMISION MIXTA', 'COMPRAS', 'COMPRAS',
       'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS',
       'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS',
       'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS',
       'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS',
       'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS',
       'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS',
       'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS',
       'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS',
       'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS',
       'COMPRAS', 'OTROS GASTOS', 'COMPRAS', 'COMPRAS', 'COMPRAS',
       'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMISION 

In [ ]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[  0   0   0   0   0   3   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   0   0   1   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   0   0   9   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   0   0  17   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   0   0   0   0   0   0   0   1   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   6   0 332   0   0   0   0   0   0   0   0   1   0   0   0
    0   0]
 [  0   0   0   0   0  10   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   0   0   7   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   0   0  20   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   0   0  21   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   0   0   9   0   0   0   0   

In [ ]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="weighted", pos_label="OTROS")
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.1920883126990466


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'OTROS') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.4289405684754522


In [ ]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="weighted", pos_label="OTROS")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.4289405684754522


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'OTROS') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# el puntaje F1 que es una combinación entre la precisión y la sensibilidad, para esto importamos f1_score.
#Calculo el Puntaje F1 del modelo
from sklearn.metrics import f1_score

puntajef1 = f1_score(y_test, y_pred, average="weighted", pos_label="OTROS")
print('Puntaje F1 del modelo:')
print(puntajef1)

Puntaje F1 del modelo:
0.26534827137441297


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'OTROS') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
